In [ ]:
# default_exp conll_reader

# A NER tagger using AllenNLP 

> API details.

In [ ]:
#export
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.instance import Instance
from allennlp.data.tokenizers import Token
from allennlp.data.fields import Field, TextField, SequenceLabelField
from overrides import overrides
from tqdm import tqdm
from typing import List, Dict, Iterator
from itertools import groupby
path_data = './data/'

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
#export
@DatasetReader.register("conll_03_reader")
class CoNLL03DatasetReader(DatasetReader): 
    def __init__(self,
                 token_indexers:Dict[str, TokenIndexer] = None,
                 lazy:bool = True) -> None: 
        """
        token_indexers are like tok2idx, basically. 
        lazy requred for anything inheriting from DatasetReader, and it is used
        to read only batches of data at once instead of putting entire dataset in memory, 
        so it is useful for large data. 
        """
        super().__init__(lazy)
        self._token_indexers = token_indexers or {'tokens': SingleIdTokenIndexer()}
        
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        # function to detect empty lines
        is_divider = lambda line: line.strip() == ''
        # open file, go through each line (read as string)
        with open(file_path, 'r') as conll_file:
            # group each sentence together in lists
            # each sentence is "sandwiched" by blank lines either side
            for divider, lines in itertools.groupby(conll_file, is_divider):
                if not divider:  # ignore blank lines
                    # string -> list
                    fields = [l.strip().split() for l in lines]
                    # switch it so that each field is a list of tokens/labels
                    # kind of like a transpose function
                    fields = [l for l in zip(*fields)]
                    # only keep the tokens and NER labels
                    tokens, _, _, ner_tags = fields
                    # convert to instance
                    yield self.text_to_instance(tokens, ner_tags)
        
    @overrides
    def text_to_instance(self, words: List[str], ner_tags: List[str]) -> Instance:
        fields: Dict[str, Field] = {}
        tokens = TextField([Token(o) for o in words],self._token_indexers )
        fields['tokens'] = tokens
        fields['label'] = SequenceLabelField(ner_tags, tokens)
        return Instance(fields)